# CS 224D Assignment #2
# Part [2]: Recurrent Neural Networks

This notebook will provide starter code, testing snippets, and additional guidance for implementing the Recurrent Neural Network Language Model (RNNLM) described in Part 2 of the handout.

Please complete parts (a), (b), and (c) of Part 2 before beginning this notebook.

In [1]:
import sys, os
from numpy import *
from matplotlib.pyplot import *
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 100

%load_ext autoreload
%autoreload 2

## (e): Implement a Recurrent Neural Network Language Model

Follow the instructions on the handout to implement your model in `rnnlm.py`, then use the code below to test.

In [2]:
from rnnlm import RNNLM
# Gradient check on toy data, for speed
random.seed(10)
wv_dummy = random.randn(10,50)
model = RNNLM(L0 = wv_dummy, U0 = wv_dummy,
              alpha=0.005, rseed=10, bptt=4)
model.grad_check(array([1,2,3]), array([2,3,4]))

rnnlm.py:55: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if U0 == None :
NOTE: temporarily setting self.bptt = len(y) = 3 to compute true gradient.
grad_check: dJ/dH error norm = 2.961e-09 [ok]
    H dims: [50, 50] = 2500 elem
grad_check: dJ/dU error norm = 4.262e-10 [ok]
    U dims: [10, 50] = 500 elem
grad_check: dJ/dL[1] error norm = 1.102e-09 [ok]
    L[1] dims: [50] = 50 elem
grad_check: dJ/dL[2] error norm = 8.874e-10 [ok]
    L[2] dims: [50] = 50 elem
grad_check: dJ/dL[3] error norm = 1.152e-09 [ok]
    L[3] dims: [50] = 50 elem
Reset self.bptt = 4


## Prepare Vocabulary and Load PTB Data

We've pre-prepared a list of the vocabulary in the Penn Treebank, along with their absolute counts and unigram frequencies. The document loader code below will "canonicalize" words and replace any unknowns with a `"UUUNKKK"` token, then convert the data to lists of indices.

In [3]:
from data_utils import utils as du
import pandas as pd

# Load the vocabulary
vocab = pd.read_table("data/lm/vocab.ptb.txt", header=None, sep="\s+",
                     index_col=0, names=['count', 'freq'], )

# Choose how many top words to keep
vocabsize = 2000
num_to_word = dict(enumerate(vocab.index[:vocabsize]))
word_to_num = du.invert_dict(num_to_word)
##
# Below needed for 'adj_loss': DO NOT CHANGE
fraction_lost = float(sum([vocab['count'][word] for word in vocab.index
                           if (not word in word_to_num) 
                               and (not word == "UUUNKKK")]))
fraction_lost /= sum([vocab['count'][word] for word in vocab.index
                      if (not word == "UUUNKKK")])
print "Retained %d words from %d (%.02f%% of all tokens)" % (vocabsize, len(vocab),
                                                             100*(1-fraction_lost))

Retained 2000 words from 38444 (84.00% of all tokens)


Load the datasets, using the vocabulary in `word_to_num`. Our starter code handles this for you, and also generates lists of lists X and Y, corresponding to input words and target words*. 

*(Of course, the target words are just the input words, shifted by one position, but it can be cleaner and less error-prone to keep them separate.)*

In [4]:
# Load the training set
docs = du.load_dataset('data/lm/ptb-train.txt')
S_train = du.docs_to_indices(docs, word_to_num)
X_train, Y_train = du.seqs_to_lmXY(S_train)

# Load the dev set (for tuning hyperparameters)
docs = du.load_dataset('data/lm/ptb-dev.txt')
S_dev = du.docs_to_indices(docs, word_to_num)
X_dev, Y_dev = du.seqs_to_lmXY(S_dev)

# Load the test set (final evaluation only)
docs = du.load_dataset('data/lm/ptb-test.txt')
S_test = du.docs_to_indices(docs, word_to_num)
X_test, Y_test = du.seqs_to_lmXY(S_test)

# Display some sample data
print " ".join(d[0] for d in docs[7])
print S_test[7]

Big investment banks refused to step up to the plate to support the beleaguered floor traders by buying big blocks of stock , traders say .
[   4  147  169  250 1879    7 1224   64    7    1    3    7  456    1    3
 1024  255   24  378  147    3    6   67    0  255  138    2    5]


## (f): Train and evaluate your model

When you're able to pass the gradient check, let's run our model on some real language!

You should randomly initialize the word vectors as Gaussian noise, i.e. $L_{ij} \sim \mathit{N}(0,0.1)$ and $U_{ij} \sim \mathit{N}(0,0.1)$; the function `random.randn` may be helpful here.

As in Part 1, you should tune hyperparameters to get a good model.

In [5]:
hdim = 100 # dimension of hidden layer = dimension of word vectors
random.seed(10)
# L0 = zeros((vocabsize, hdim)) # replace with random init, 
                              # or do in RNNLM.__init__()
L0 = sqrt(0.1) * random.randn(vocabsize, hdim)   # Gaussian noise
# test parameters; you probably want to change these
model = RNNLM(L0, U0 = L0, alpha=0.1, rseed=10, bptt=1)

from nn.base import NNBase
# Gradient check is going to take a *long* time here
# since it's quadratic-time in the number of parameters.
# run at your own risk... (but do check this!)
# model.grad_check(array([1,2,3]), array([2,3,4]))

In [6]:
#### YOUR CODE HERE ####
##
# Pare down to a smaller dataset, for speed
# (optional - recommended to not do this for your final model)
# using a small vocabulary of 2000-5000 words.
ntrain = len(Y_train)
print ntrain
num = 5000
X = X_train[:num]
Y = Y_train[:num]

56522


In [ ]:
# tune the hyperparameter, batchsize, alpha, bptt
batchsize = [5,10,15,20,25,50] # too slow if increasing the batch size. so set batchsize = 5

for bs in batchsize:
    print "===== Batch size %d: =====" % bs
    model = RNNLM(L0, U0 = L0, alpha=0.1, rseed=10, bptt=1)
    schedule = NNBase.randomiter(5*num/bs, num, bs)
    model.train_sgd(X=X, y=Y, idxiter=schedule, printevery=500, costevery=500)
    print "===== cross-entropy loss on the dev set %f: =====" % model.compute_mean_loss(X_dev, Y_dev)

#### END YOUR CODE ####

56522
===== Batch size 5: =====
Begin SGD...
  Seen 0 in 0.00 s
  [0]: mean loss 8.41326
  Seen 500 in 80.70 s
  [500]: mean loss 4.93806
  Seen 1000 in 166.66 s
  [1000]: mean loss 4.764
  Seen 1500 in 250.21 s
  [1500]: mean loss 4.69337
  Seen 2000 in 328.40 s
  [2000]: mean loss 4.58571
  Seen 2500 in 406.14 s
  [2500]: mean loss 4.54951
  Seen 3000 in 485.29 s
  [3000]: mean loss 4.38956
  Seen 3500 in 563.50 s
  [3500]: mean loss 4.34754
  Seen 4000 in 642.65 s
  [4000]: mean loss 4.39601
  Seen 4500 in 722.28 s
  [4500]: mean loss 4.25939
  [5000]: mean loss 4.20713
SGD complete: 5000 examples in 830.90 seconds.
===== cross-entropy loss on the dev set 4.438971: =====
===== Batch size 10: =====
Begin SGD...
  Seen 0 in 0.00 s
  [0]: mean loss 8.41326
  Seen 500 in 131.99 s
  [500]: mean loss 5.17122
  Seen 1000 in 262.09 s
  [1000]: mean loss 5.89807
  Seen 1500 in 398.24 s
  [1500]: mean loss 4.54358
  Seen 2000 in 550.13 s
  [2000]: mean loss 4.49413
  [2500]: mean loss 4.40919

In [7]:
# tune the hyperparameter, batchsize, alpha, bptt
bs = 5
alphas = [0.1,0.01,0.005, 0.001, 0.0005, 0.0001] 

for a in alphas:
    print "===== Alpha %f: =====" % a
    model = RNNLM(L0, U0 = L0, alpha=a, rseed=10, bptt=1)
    schedule = NNBase.randomiter(5*num/bs, num, bs)
    model.train_sgd(X=X, y=Y, idxiter=schedule, printevery=500, costevery=500)
    print "===== cross-entropy loss on the dev set %f: =====" % model.compute_mean_loss(X_dev, Y_dev)

#### END YOUR CODE ####

===== Alpha 0.100000: =====
Begin SGD...
  Seen 0 in 0.00 s
  [0]: mean loss 8.41326
  Seen 500 in 85.04 s
  [500]: mean loss 4.93806
  Seen 1000 in 164.46 s
  [1000]: mean loss 4.764
  Seen 1500 in 244.51 s
  [1500]: mean loss 4.69337
  Seen 2000 in 324.98 s
  [2000]: mean loss 4.58571
  Seen 2500 in 409.26 s
  [2500]: mean loss 4.54951
  Seen 3000 in 487.99 s
  [3000]: mean loss 4.38956
  Seen 3500 in 569.07 s
  [3500]: mean loss 4.34754
  Seen 4000 in 648.35 s
  [4000]: mean loss 4.39601
  Seen 4500 in 729.29 s
  [4500]: mean loss 4.25939
  [5000]: mean loss 4.20713
SGD complete: 5000 examples in 842.18 seconds.
===== cross-entropy loss on the dev set 4.438971: =====
===== Alpha 0.010000: =====
Begin SGD...
  Seen 0 in 0.00 s
  [0]: mean loss 8.41326
  Seen 500 in 81.80 s
  [500]: mean loss 5.06165
  Seen 1000 in 163.67 s
  [1000]: mean loss 4.97185
  Seen 1500 in 245.72 s
  [1500]: mean loss 4.97417
  Seen 2000 in 325.68 s
  [2000]: mean loss 4.82507
  Seen 2500 in 406.69 s
  [2500

In [8]:
bs = 5
alpha = 0.1
time_steps = [2,3,4,5]

for t in time_steps:
    print "===== Time_steps %f: =====" % t
    model = RNNLM(L0, U0 = L0, alpha=alpha, rseed=10, bptt=t)
    schedule = NNBase.randomiter(5*num/bs, num, bs)
    model.train_sgd(X=X, y=Y, idxiter=schedule, printevery=1000, costevery=1000)
    print "===== cross-entropy loss on the dev set %f: =====" % model.compute_mean_loss(X_dev, Y_dev)

#### END YOUR CODE ####

===== Time_steps 2.000000: =====
Begin SGD...
  Seen 0 in 0.00 s
  [0]: mean loss 8.41326
  Seen 1000 in 216.20 s
  [1000]: mean loss 4.77212
  Seen 2000 in 494.17 s
  [2000]: mean loss 4.61309
  Seen 3000 in 719.69 s
  [3000]: mean loss 4.47269
  Seen 4000 in 938.57 s
  [4000]: mean loss 4.5085
  [5000]: mean loss 4.33139
SGD complete: 5000 examples in 1209.38 seconds.
===== cross-entropy loss on the dev set 4.507844: =====
===== Time_steps 3.000000: =====
Begin SGD...
  Seen 0 in 0.00 s
  [0]: mean loss 8.41326
  Seen 1000 in 268.90 s
  [1000]: mean loss 4.89763
  Seen 2000 in 544.39 s
  [2000]: mean loss 4.6255
  Seen 3000 in 772.01 s
  [3000]: mean loss 4.48653
  Seen 4000 in 996.72 s
  [4000]: mean loss 4.53041
  [5000]: mean loss 4.33976
SGD complete: 5000 examples in 1264.24 seconds.
===== cross-entropy loss on the dev set 4.532734: =====
===== Time_steps 4.000000: =====
Begin SGD...
  Seen 0 in 0.00 s
  [0]: mean loss 8.41326
  Seen 1000 in 224.66 s
  [1000]: mean loss 4.72919


In [ ]:
## Evaluate cross-entropy loss on the dev set,
## then convert to perplexity for your writeup
model = RNNLM(L0, U0 = L0, alpha=0.1, rseed=10, bptt=1)
schedule = NNBase.randomiter(len(Y_train), len(Y_train), 5)
model.train_sgd(X=X_train, y=Y_train, idxiter=schedule, printevery=5000, costevery=5000)
print "===== cross-entropy loss on the dev set %f: =====" % model.compute_mean_loss(X_dev, Y_dev)

The performance of the model is skewed somewhat by the large number of `UUUNKKK` tokens; if these are 1/6 of the dataset, then that's a sizeable fraction that we're just waving our hands at. Naively, our model gets credit for these that's not really deserved; the formula below roughly removes this contribution from the average loss. Don't worry about how it's derived, but do report both scores - it helps us compare across models with different vocabulary sizes.

In [21]:
## DO NOT CHANGE THIS CELL ##
# Report your numbers, after computing dev_loss above.
def adjust_loss(loss, funk, q, mode='basic'):
    if mode == 'basic':
        # remove freebies only: score if had no UUUNKKK
        return (loss + funk*log(funk))/(1 - funk)
    else:
        # remove freebies, replace with best prediction on remaining
        return loss + funk*log(funk) - funk*log(q)
# q = best unigram frequency from omitted vocab
# this is the best expected loss out of that set
q = vocab.freq[vocabsize] / sum(vocab.freq[vocabsize:])
print "Unadjusted: %.03f" % exp(dev_loss)
print "Adjusted for missing vocab: %.03f" % exp(adjust_loss(dev_loss, fraction_lost, q))

### Save Model Parameters

In [16]:
##
# Save to .npy files; should only be a few MB total
assert(min(model.sparams.L.shape) <= 100) # don't be too big
assert(max(model.sparams.L.shape) <= 5000) # don't be too big
save("rnnlm.L.npy", model.sparams.L)
save("rnnlm.U.npy", model.params.U)
save("rnnlm.H.npy", model.params.H)

## (g): Generating Data

Once you've trained your model to satisfaction, let's use it to generate some sentences!

Implement the `generate_sequence` function in `rnnlm.py`, and call it below.

In [19]:
def seq_to_words(seq):
    return [num_to_word[s] for s in seq]
    
seq, J = model.generate_sequence(word_to_num["<s>"], 
                                 word_to_num["</s>"], 
                                 maxlen=100)
print J
# print seq
print " ".join(seq_to_words(seq))

**BONUS:** Use the unigram distribution given in the `vocab` table to fill in any `UUUNKKK` tokens in your generated sequences with words that we omitted from the vocabulary. You'll want to use `list(vocab.index)` to get a list of words, and `vocab.freq` to get a list of corresponding frequencies.

In [20]:
# Replace UUUNKKK with a random unigram,
# drawn from vocab that we skipped
from nn.math import MultinomialSampler, multinomial_sample
def fill_unknowns(words):
    #### YOUR CODE HERE ####
    ret = words # do nothing; replace this
    

    #### END YOUR CODE ####
    return ret
    
print " ".join(fill_unknowns(seq_to_words(seq)))